In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_path = "/content/drive/MyDrive/price_paid_records.csv"


In [5]:
import pandas as pd
import cudf
import time
import numpy as np


In [6]:
#Question-1

In [7]:
# Load with pandas using full path
start_time = time.time()
pdf = pd.read_csv(file_path)
pandas_load_time = time.time() - start_time
print(f"Pandas load time: {pandas_load_time:.4f} seconds")

# Load with cuDF using same path
start_time = time.time()
gdf = cudf.read_csv(file_path)
cudf_load_time = time.time() - start_time
print(f"cuDF load time: {cudf_load_time:.4f} seconds")


Pandas load time: 70.7017 seconds
cuDF load time: 11.4322 seconds


In [8]:
# Pandas
start_time = time.time()
print("Pandas .info():")
pdf.info()
print("\nPandas .describe():")
print(pdf.describe())
pandas_info_desc_time = time.time() - start_time
print(f"Pandas .info() + .describe(): {pandas_info_desc_time:.4f} seconds")

# cuDF
start_time = time.time()
print("\ncuDF .info():")
gdf.info()
print("\ncuDF .describe():")
print(gdf.describe())
cudf_info_desc_time = time.time() - start_time
print(f"cuDF .info() + .describe(): {cudf_info_desc_time:.4f} seconds")


Pandas .info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22489348 entries, 0 to 22489347
Data columns (total 11 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   Transaction unique identifier      object
 1   Price                              int64 
 2   Date of Transfer                   object
 3   Property Type                      object
 4   Old/New                            object
 5   Duration                           object
 6   Town/City                          object
 7   District                           object
 8   County                             object
 9   PPDCategory Type                   object
 10  Record Status - monthly file only  object
dtypes: int64(1), object(10)
memory usage: 1.8+ GB

Pandas .describe():
              Price
count  2.248935e+07
mean   1.782442e+05
std    3.903677e+05
min    1.000000e+00
25%    7.500000e+04
50%    1.300000e+05
75%    2.100000e+05
max    9.890000e+07
Pandas

In [9]:
# Pandas
start_time = time.time()
pdf_cleaned = pdf.dropna()
pandas_dropna_time = time.time() - start_time
print(f"Pandas dropna: {pandas_dropna_time:.4f} seconds")

# cuDF
start_time = time.time()
gdf_cleaned = gdf.dropna()
cudf_dropna_time = time.time() - start_time
print(f"cuDF dropna: {cudf_dropna_time:.4f} seconds")


Pandas dropna: 13.6657 seconds
cuDF dropna: 0.0553 seconds


In [10]:
# Clean column names (remove extra spaces if any)
pdf_cleaned.columns = pdf_cleaned.columns.str.strip()
gdf_cleaned.columns = gdf_cleaned.columns.str.strip()

# Display column names to identify the correct price column
print("Pandas Columns:\n", pdf_cleaned.columns.tolist())
print("cuDF Columns:\n", gdf_cleaned.columns.tolist())


Pandas Columns:
 ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']
cuDF Columns:
 ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']


In [11]:
# Clean column names of extra whitespace
pdf_cleaned.columns = pdf_cleaned.columns.str.strip()
gdf_cleaned.columns = gdf_cleaned.columns.str.strip()

# Print all column names
print("📌 Columns in pandas DataFrame:")
print(pdf_cleaned.columns.tolist())


📌 Columns in pandas DataFrame:
['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']


In [12]:
import numpy as np
import cupy as cp  # Required for GPU operations in cuDF
import time

# Clean column names in case of extra spaces
pdf_cleaned.columns = pdf_cleaned.columns.str.strip()
gdf_cleaned.columns = gdf_cleaned.columns.str.strip()

# 🔹 Pandas log transformation
start_time = time.time()
pdf_cleaned['log_price'] = np.log(pdf_cleaned['Price'])
pandas_log_time = time.time() - start_time
print(f"Pandas log(Price): {pandas_log_time:.4f} seconds")

# 🔹 cuDF log transformation
start_time = time.time()
# Convert cuDF Series to CuPy array and apply log
gdf_cleaned['log_price'] = cp.log(gdf_cleaned['Price'].to_cupy())
cudf_log_time = time.time() - start_time
print(f"cuDF log(Price): {cudf_log_time:.4f} seconds")


Pandas log(Price): 0.2706 seconds
cuDF log(Price): 0.6270 seconds


In [13]:
print(pdf_cleaned.columns.tolist())


['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only', 'log_price']


In [15]:
from datetime import datetime, timedelta

# Define the date column name and cutoff date
date_column = 'Date of Transfer'
cutoff_date = datetime.now() - timedelta(days=5*365) # Approximately 5 years ago

# 🟩 STEP 1: Drop nulls first in the date column (if needed)
gdf_cleaned = gdf_cleaned.dropna(subset=[date_column])

# 🟩 STEP 2: Convert to datetime (no 'errors' param)
gdf_cleaned[date_column] = cudf.to_datetime(gdf_cleaned[date_column])

# 🟩 STEP 3: Filter records within the last 5 years
start_time = time.time()
gdf_recent = gdf_cleaned[gdf_cleaned[date_column] >= cutoff_date]
cudf_filter_time = time.time() - start_time
print(f"cuDF filter (last 5 years): {cudf_filter_time:.4f} seconds")

cuDF filter (last 5 years): 0.0262 seconds


In [16]:
# 🟩 STEP 1: Drop nulls first in the date column (if needed)
gdf_cleaned = gdf_cleaned.dropna(subset=[date_column])

# 🟩 STEP 2: Convert to datetime (no 'errors' param)
gdf_cleaned[date_column] = cudf.to_datetime(gdf_cleaned[date_column])

# 🟩 STEP 3: Filter records within the last 5 years
start_time = time.time()
gdf_recent = gdf_cleaned[gdf_cleaned[date_column] >= cutoff_date]
cudf_filter_time = time.time() - start_time
print(f"cuDF filter (last 5 years): {cudf_filter_time:.4f} seconds")


cuDF filter (last 5 years): 0.0071 seconds


In [ ]:
#Question-2

In [1]:
import pandas as pd
import numpy as np
import time

# Step 1: Load your data again
file_path = "/content/drive/MyDrive/price_paid_records.csv"
pdf = pd.read_csv(file_path)

# Step 2: Clean column names
pdf.columns = pdf.columns.str.strip()

# Step 3: Drop nulls
pdf_cleaned = pdf.dropna()

# Step 4: Fix price column and add log_price
pdf_cleaned['Price'] = pdf_cleaned['Price'].astype(float)  # Ensure numeric
pdf_cleaned['log_price'] = np.log(pdf_cleaned['Price'])

# Done! Now pdf_cleaned is back in memory


In [2]:
print(pdf_cleaned.columns.tolist())


['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only', 'log_price']


In [3]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Property Type', 'Town/City', 'County']
label_encoders = {}

for col in categorical_cols:
    pdf_cleaned[col] = pdf_cleaned[col].astype(str).str.strip()
    le = LabelEncoder()
    pdf_cleaned[col] = le.fit_transform(pdf_cleaned[col])
    label_encoders[col] = le


In [4]:
median_price = pdf_cleaned['Price'].median()
pdf_cleaned['price_category'] = (pdf_cleaned['Price'] > median_price).astype(int)


In [5]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
features = ['Property Type', 'Town/City', 'County', 'log_price']
X = pdf_cleaned[features]
y = pdf_cleaned['price_category']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [6]:
# Sample 50,000 rows for quick testing
sampled_df = pdf_cleaned.sample(n=50000, random_state=42)

# Redefine X and y
features = ['Property Type', 'Town/City', 'County', 'log_price']
X = sampled_df[features]
y = sampled_df['price_category']


In [8]:
from sklearn.model_selection import train_test_split

# Redefine X and y using sampled_df
features = ['Property Type', 'Town/City', 'County', 'log_price']
X = sampled_df[features]
y = sampled_df['price_category']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import time

# Train
start_time = time.time()
rf_sklearn = RandomForestClassifier(n_estimators=20, max_depth=10, random_state=42)
rf_sklearn.fit(X_train, y_train)
sklearn_train_time = time.time() - start_time

# Predict
start_time = time.time()
y_pred_sklearn = rf_sklearn.predict(X_test)
sklearn_predict_time = time.time() - start_time

# Evaluate
sklearn_acc = accuracy_score(y_test, y_pred_sklearn)
sklearn_f1 = f1_score(y_test, y_pred_sklearn)

print("✅ Scikit-learn done")


✅ Scikit-learn done


In [10]:
import cudf

# Convert the sampled pandas DataFrame to cuDF
gdf = cudf.from_pandas(sampled_df)


In [12]:
import cudf
from cuml.model_selection import train_test_split as cu_train_test_split

# Convert the sampled pandas DataFrame to cuDF
gdf = cudf.from_pandas(sampled_df)

# cuML train-test split
X_cu = gdf[features]
y_cu = gdf['price_category']

X_train_cu, X_test_cu, y_train_cu, y_test_cu = cu_train_test_split(
    X_cu, y_cu, test_size=0.2, random_state=42
)

In [14]:
from cuml.ensemble import RandomForestClassifier as cuRF
import time

# Train cuML model
start_time = time.time()
rf_cuml = cuRF(n_estimators=20, max_depth=10, random_state=42)
rf_cuml.fit(X_train_cu, y_train_cu)
cuml_train_time = time.time() - start_time

# Predict
start_time = time.time()
y_pred_cuml = rf_cuml.predict(X_test_cu)
cuml_predict_time = time.time() - start_time

# Evaluate (evaluation is done in the next cell)
print("✅ cuML done")

✅ cuML done


In [15]:
from sklearn.metrics import accuracy_score, f1_score

# Evaluate
cuml_acc = accuracy_score(y_test_cu.to_numpy(), y_pred_cuml.to_numpy())
cuml_f1 = f1_score(y_test_cu.to_numpy(), y_pred_cuml.to_numpy())


In [ ]:
#Question-3

In [1]:
import pandas as pd
import numpy as np

# Load the original CSV file
file_path = "/content/drive/MyDrive/price_paid_records.csv"
pdf = pd.read_csv(file_path)

# Strip column names
pdf.columns = pdf.columns.str.strip()

# Drop nulls
pdf_cleaned = pdf.dropna()

# Ensure price is float
pdf_cleaned['Price'] = pdf_cleaned['Price'].astype(float)

# Add log_price column
pdf_cleaned['log_price'] = np.log(pdf_cleaned['Price'])


In [2]:
sampled_df = pdf_cleaned.sample(n=50000, random_state=42)


In [3]:
df = sampled_df.copy()


In [4]:
df['postcode'] = df['Transaction unique identifier'].str.extract(r'([A-Z]{1,2}\d{1,2})', expand=False)


In [5]:
print(df.columns)
print(df[['Transaction unique identifier', 'postcode']].head())


Index(['Transaction unique identifier', 'Price', 'Date of Transfer',
       'Property Type', 'Old/New', 'Duration', 'Town/City', 'District',
       'County', 'PPDCategory Type', 'Record Status - monthly file only',
       'log_price', 'postcode'],
      dtype='object')
                   Transaction unique identifier postcode
13893637  {6146E264-E0D9-4C53-ACC8-48DB3954F80B}      E26
18522801  {26EBD75A-D90F-411C-85E5-4D56F0F66484}     BD75
7259339   {E700C723-9426-4924-8D3F-1730EC3B2BCC}      E70
2129082   {677E0E46-8E8F-4560-AD93-07F72D5AE6D5}       E0
3112615   {E2387F76-24EC-4A7E-8A27-220E500F0DC2}      E23


In [6]:
from itertools import combinations

# Generate edges by grouping
def generate_edges(group_col):
    edges = []
    for _, group in df.groupby(group_col):
        postcodes = group['postcode'].unique()
        edges.extend(combinations(postcodes, 2))
    return edges

# Create edges from both town and county
town_edges = generate_edges('Town/City')
county_edges = generate_edges('County')

# Combine and create DataFrame
all_edges = set(town_edges + county_edges)
edges_df = pd.DataFrame(list(all_edges), columns=['src', 'dst'])
